# AirBnB Dataset from Boston and Seattle

Using the AirBnb dataset provided from Udacity course for the cities of Seattle (https://www.kaggle.com/airbnb/seattle/data) and Boston(https://www.kaggle.com/airbnb/boston), the objective here is to explore it answering the following questions.

## Questions to answer:
- Which is the period with less available Airbnbs in Seattle? And in Boston?
- Which city has the average higher price for AirBnb?
- Which is the most rented property in Seattle? And for Boston?
- Which is the most important feature regarding the AirBnb price?

### Importing libraries

In [1]:
# importing libraries
import pandas as pd 
import numpy as np 
from glob import glob
import os
import plotly.express as px
from tqdm import tqdm

In [20]:
# setting relatives paths to use the folder structure
# - basic folder
# -- scripts
# -- raw_data
RAW_DATA_PATH = os.path.join("..", "raw_data")

# sets a list of the cities included in the dataset
CITIES = ['seattle', 'boston']

### Importing data

In [21]:
# gather filenames
def get_file_names(path, extension):
    '''
    Using the base path and extension, returns all filenames in the path with that extension

    Parameters:
    path (string): the base path for seeking the files. It can be relative
    extension (string): the extension for seeking the files

    Returns:
    list of filenames in that folder and with that filename
    
    '''
    return glob(os.path.join(path, "*" + extension))

files = get_file_names(RAW_DATA_PATH, '.csv')
files

['..\\raw_data\\calendar_boston.csv',
 '..\\raw_data\\calendar_seattle.csv',
 '..\\raw_data\\listings_boston.csv',
 '..\\raw_data\\listings_seattle.csv',
 '..\\raw_data\\reviews_boston.csv',
 '..\\raw_data\\reviews_seattle.csv']

The data is divided in three datasets for each city (6 datasets in total), so we will gather the datasets for both cities and merge them in three datasets with a column showing us which city the data is about. At the end, we will have 3 datasets, as follows:
- calendar_df: data about the occupation of the Airbnbs properties all over years
- listing_df: data about the Airbnb properties
- reviews_df: data with informations about the reviews about each property. This dataset will be not used in this exploratory data analysis

## Assess data

In [22]:
# opens each file and merges it with the appropriete dataset
def load_and_merge_files(filenames):
    '''
    Loads and merge datasets from different cities into three different datasets, listing_df, reviews_df and calendar_df

    Parameters:
    filenames (list): List of filenames for the datasets to be loaded

    Returns:
    listing_df (dataframe): Dataframe containing the informations about the properties
    reviews_df (dataframe): Dataframe containing the reviews
    calendar_df (dataframe): Dataframe containing the calendar data about the properties
    '''
    listing_dfs = []
    calendar_dfs = []
    reviews_dfs = []

    for file in files:
        city = file.split("_")[2].split(".")[0]
        df_aux = pd.read_csv(file)
        df_aux['city'] = city
        if 'listing' in file:
            listing_dfs.append(df_aux)
        elif 'calendar' in file:
            calendar_dfs.append(df_aux)
        elif 'reviews' in file:
            reviews_dfs.append(df_aux)

    listing_df = pd.concat(listing_dfs, sort=False)
    reviews_df = pd.concat(reviews_dfs, sort=False) 
    calendar_df = pd.concat(calendar_dfs, sort=False)

    return listing_df, reviews_df, calendar_df

In [23]:
# lets use our function to gather and merge files into 3 different datasets
listing_df, reviews_df, calendar_df = load_and_merge_files(files)

In [24]:
# looking at listing_df
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7403 entries, 0 to 3817
Data columns (total 95 columns):
id                                  7403 non-null int64
listing_url                         7403 non-null object
scrape_id                           7403 non-null int64
last_scraped                        7403 non-null object
name                                7403 non-null object
summary                             7083 non-null object
space                               5777 non-null object
description                         7403 non-null object
experiences_offered                 7403 non-null object
neighborhood_overview               4956 non-null object
notes                               3822 non-null object
transit                             5179 non-null object
access                              2096 non-null object
interaction                         2031 non-null object
house_rules                         2393 non-null object
thumbnail_url                       6484

In [25]:
# lets do a quick vizualization on the data provided
listing_df.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [26]:
# lets convert the price value in a float value (originally it is interpretaded as a string)
listing_df['price'] = listing_df['price'].apply(lambda row: str(row).replace("$", "").replace(",", "")).astype(float)

In [27]:
# lets take a look on reviews dataframe
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153124 entries, 0 to 84848
Data columns (total 7 columns):
listing_id       153124 non-null int64
id               153124 non-null int64
date             153124 non-null object
reviewer_id      153124 non-null int64
reviewer_name    153124 non-null object
comments         153053 non-null object
city             153124 non-null object
dtypes: int64(3), object(4)
memory usage: 9.3+ MB


In [28]:
# lets view a little about reviews_df
reviews_df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments,city
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...,boston
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...,boston
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...,boston
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...,boston
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...,boston


In [29]:
# and lets take a look on calendar_df
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2702460 entries, 0 to 1393569
Data columns (total 5 columns):
listing_id    int64
date          object
available     object
price         object
city          object
dtypes: int64(1), object(4)
memory usage: 123.7+ MB


In [30]:
calendar_df.head()

,listing_id,date,available,price,city
0,12147973,2017-09-05,f,NaN,boston
1,12147973,2017-09-04,f,NaN,boston
2,12147973,2017-09-03,f,NaN,boston
3,12147973,2017-09-02,f,NaN,boston
4,12147973,2017-09-01,f,NaN,boston


Once we've gathered all the necessary data and understood better it, lets move to the first question we are trying to answer

# Question #1: Which is the period with less available Airbnbs in Seattle? And in Boston?

For answer this question, we should use the calendar_df dataset, because it has the dates and the prices required by our analysis. But first, we must clean it and adjust column types

### Clean dataset

In [31]:
# lets convert the columns to a more suitable type
calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%Y-%m-%d')
calendar_df['price'] = calendar_df['price'].apply(lambda row: str(row).replace("$", "").replace(",", "")).astype(float)
calendar_df = pd.get_dummies(calendar_df, columns=['available'], drop_first=True)
calendar_df.describe()

,listing_id,price,available_t
count,2.702460e+06,1.577579e+06,2.702460e+06
mean,6.950805e+06,1.626028e+02,5.837567e-01
std,4.052440e+06,1.437014e+02,4.929350e-01
min,3.335000e+03,1.000000e+01,0.000000e+00
25%,3.865313e+06,7.900000e+01,0.000000e+00
50%,7.035369e+06,1.200000e+02,1.000000e+00
75%,9.504211e+06,2.000000e+02,1.000000e+00
max,1.493346e+07,7.163000e+03,1.000000e+00


In [32]:
# lets fill the NaN values in the 'price' column with the 'price' mean values from the 'listing_df' dataset (because there we have all the prices)
# this step may take a while
calendar_df_prices = calendar_df.copy()

for listing in tqdm(calendar_df_prices['listing_id'].unique()):
    calendar_df_prices.loc[calendar_df_prices['listing_id'] == listing, 'price'] = listing_df[listing_df['id'] == listing]['price'].mean()

100%|██████████| 7403/7403 [03:47<00:00, 32.51it/s]


In [33]:
# lets check if the process worked well, yes it was
calendar_df_prices.isnull().sum()

listing_id     0
date           0
price          0
city           0
available_t    0
dtype: int64

In [43]:
# to understand how is the availability over the year for city, we should calculate the mean of availability for all properties, in which city
# this way, we can groupby our dataset per city and per date, to get the mean availability to each date

mean_avail_prices = calendar_df_prices.groupby(by=['city', 'date']).mean()
mean_avail_prices.head()

listing_id      price  available_t
city   date                                            
boston 2016-09-06  8.442118e+06  173.93865     0.158951
       2016-09-07  8.442118e+06  173.93865     0.259900
       2016-09-08  8.442118e+06  173.93865     0.278583
       2016-09-09  8.442118e+06  173.93865     0.273564
       2016-09-10  8.442118e+06  173.93865     0.266592

In [44]:
def ungroup_dataframe(grouped_dataframe, cities):
    '''
    Removes the dataframe MultiIndex of Cities and other features, 
    
    It removes the MultiIndex, removing the city from the Index by creating a column named City and putting the city values in it. 

    Parameters:
    grouped_dataframe (dataframe): a dataframe with MultiIndex, and the first level of index as Cities
    cities (list): A list of cities where the dataset came from

    Returns:
    Dataframe: which has a single index

    '''
    ungrouped_dataframe = grouped_dataframe.copy()
    ungrouped_dataframe['city'] = ""

    for city in cities:
        ungrouped_dataframe.loc[city, 'city'] = city

    ungrouped_dataframe = ungrouped_dataframe.droplevel(level=0)

    return ungrouped_dataframe

In [45]:
# yes, it worked
# now, it is necessary to remove the MultiIndex to easy plot the data
# So, we created again the city collumn and put the city information in it
# And removed the city from our index
# It is easily done by our ungroup_dataframe function

mean_avail_prices = ungroup_dataframe(mean_avail_prices, CITIES)
mean_avail_prices.head()

,listing_id,price,available_t,city
date,,,,
2016-09-06,8.442118e+06,173.93865,0.158951,boston
2016-09-07,8.442118e+06,173.93865,0.259900,boston
2016-09-08,8.442118e+06,173.93865,0.278583,boston
2016-09-09,8.442118e+06,173.93865,0.273564,boston
2016-09-10,8.442118e+06,173.93865,0.266592,boston


### Analyze and Vizualize

In [46]:
# Using Plotly Express, which is an interactive data vizualition tool, we can plot our results
fig = px.line(mean_avail_prices, x=mean_avail_prices.index, y="available_t", color='city')
fig.show()

Seattle clearly has a occupation lower than Boston, as we can realize from the plot above. For Seattle, the period with the most of vacancy Airbnbs are in the months of April, and from October 1st to January 1st, 2017. For Boston, which has a much higher occupation, the period with the most vacancy is in December of 2016. Although the data is not perfect for the period, with less overlapping, we could realize easily that Boston Airbnb properties tend to be more occupated.

## Question #2: Which city has the average higher price for AirBnb?

Let's move to the second question, using the 'calendar_df', once we fulfilled the prices in it. Additionally, we can use the same groupby we already used and only plot the results.

In [47]:
fig = px.line(mean_avail_prices, x=mean_avail_prices.index, y="price", color='city')
fig.show()

From the plot above, we can realize that Boston has higher price than Seattle, which colaborates with the finding that Boston has less vacancy in Airbnbs than Seattle (supply and demand)

## Question #3: Which is the most rented property in Seattle? And for Boston?

To answer this question, we will still use the calendar_df dataset and verify the lower value for the 'availability_t' column. First, lets vizualize the data

In [48]:
calendar_df_prices.head()

,listing_id,date,price,city,available_t
0,12147973,2017-09-05,250.0,boston,0
1,12147973,2017-09-04,250.0,boston,0
2,12147973,2017-09-03,250.0,boston,0
3,12147973,2017-09-02,250.0,boston,0
4,12147973,2017-09-01,250.0,boston,0


In [49]:
# to answer this question, we should groupby the data by city and property (listing_id), and calculate the average of available time for each property

availability_per_listing = calendar_df_prices.groupby(by=['city', 'listing_id']).mean()

# lets undo the MultiIndex again using our ungroup_dataframe function
availability_per_listing = ungroup_dataframe(availability_per_listing, CITIES)

# sort and vizualize the results
availability_per_listing.sort_values(by='available_t', ascending=True, inplace=True)
availability_per_listing.head(10)

,price,available_t,city
listing_id,,,
13379155,52.0,0.0,boston
3457722,250.0,0.0,boston
3458030,500.0,0.0,boston
13980127,350.0,0.0,boston
3521944,119.0,0.0,boston
8696236,105.0,0.0,boston
8696200,103.0,0.0,boston
6091789,200.0,0.0,seattle
8631257,50.0,0.0,boston


As we could vizualize the data in the table above, there are more than one property fully occupied all over the property. So, lets change a little our approach. Lets count how many properties are fully occupied all over the period for each city.

### For Seattle

In [50]:
# Lets count
len(availability_per_listing[(availability_per_listing['city'] == 'seattle') & (availability_per_listing['available_t'] == 0.0)])

95

So, for Seattle, we find out that 95 properties there was fully located for all the period in the dataset, from a total of 3818 properties

In [51]:
# Total number of properties from Seattle
len(availability_per_listing[availability_per_listing['city'] == 'seattle'])

3818

### For Boston

In [52]:
# Lets count
len(availability_per_listing[(availability_per_listing['city'] == 'boston') & (availability_per_listing['available_t'] == 0.0)])

679

For Boston, we found out that 679 properties were busy for all the period, from the 3585 properties. This colaborates with our previous finding that Boston has a much higher occupation than Seattle

In [53]:
# Total number of properties for Boston
len(availability_per_listing[availability_per_listing['city'] == 'boston'])

3585

And the next plot, shows us the availability according to the price to each city. From it, we can realize that Boston has much more properties not available and, in general, the AirBnB properties of our dataset has a lower price (<$1000). Also, Boston has the highest prices properties, which colaborates with our previous finding.

In [54]:
fig = px.scatter(availability_per_listing, x='available_t', y="price", color='city')
fig.show()

Lets go further and plot an histogram of availabilities for Seattle

In [55]:
fig = px.histogram(availability_per_listing[availability_per_listing['city'] == 'seattle'], x="available_t")
fig.show()

Histogram of availabilities for Boston

In [56]:
fig = px.histogram(availability_per_listing[availability_per_listing['city'] == 'boston'], x="available_t")
fig.show()

So, its clear to see that Boston has a lot of properties rented during all over the period

## Question #4: Which is the most important feature regarding the AirBnb price?

To answer this question, let's back to the listing dataset, where we can do a regression and find out the most important features in it.

In [57]:
# lets remember which data is available
listing_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [58]:
# available columns
listing_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [59]:
#features that makes sense to be in the model
features = ['price', 'experiences_offered',  'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_has_profile_pic', 'city',             'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'square_feet', 'security_deposit', 'zipcode',                    'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
            'maximum_nights', 'number_of_reviews', 'review_scores_value']

price_predict = listing_df[features].copy()
price_predict.head()

,price,experiences_offered,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,city,property_type,room_type,...,square_feet,security_deposit,zipcode,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_value
0,250.0,none,NaN,NaN,NaN,f,t,boston,House,Entire home/apt,...,NaN,NaN,02131,$35.00,1,$0.00,2,1125,0,NaN
1,65.0,none,within an hour,100%,100%,f,t,boston,Apartment,Private room,...,NaN,$95.00,02131,$10.00,0,$0.00,2,15,36,9.0
2,65.0,none,within a few hours,100%,88%,t,t,boston,Apartment,Private room,...,NaN,NaN,02131,NaN,1,$20.00,3,45,41,10.0
3,75.0,none,within a few hours,100%,50%,f,t,boston,House,Private room,...,NaN,$100.00,NaN,$50.00,2,$25.00,1,1125,1,10.0
4,79.0,none,within an hour,100%,100%,t,t,boston,House,Private room,...,NaN,NaN,02131,$15.00,1,$0.00,2,31,29,10.0


In [60]:
# lets verify how many values there are in our columns
price_predict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7403 entries, 0 to 3817
Data columns (total 25 columns):
price                   7403 non-null float64
experiences_offered     7403 non-null object
host_response_time      6409 non-null object
host_response_rate      6409 non-null object
host_acceptance_rate    6159 non-null object
host_is_superhost       7401 non-null object
host_has_profile_pic    7401 non-null object
city                    7403 non-null object
property_type           7399 non-null object
room_type               7403 non-null object
accommodates            7403 non-null int64
bathrooms               7373 non-null float64
bedrooms                7387 non-null float64
beds                    7393 non-null float64
bed_type                7403 non-null object
square_feet             153 non-null float64
security_deposit        3208 non-null object
zipcode                 7358 non-null object
cleaning_fee            5266 non-null object
guests_included         7403 non-nu

In [61]:
# Although 'square_feet' appears to be an important feature to our model, we have many values null, so we shall discard it
price_predict.drop('square_feet', axis=1, inplace=True)

In [62]:
# lets transform some numerical columns interpreted as strings to float
price_predict['host_acceptance_rate'] = price_predict['host_acceptance_rate'].apply(lambda row: str(row).replace("%", "").replace("none", "0"))
price_predict['host_response_rate'] = price_predict['host_response_rate'].apply(lambda row: str(row).replace("%", "").replace("none", "0"))
price_predict[['host_acceptance_rate', 'host_response_rate']] = price_predict[['host_acceptance_rate', 'host_response_rate']].astype(float)

price_predict['security_deposit'] = price_predict['security_deposit'].apply(lambda row: str(row).replace("$", "").replace(",", "")).astype(float)
price_predict['cleaning_fee'] = price_predict['cleaning_fee'].apply(lambda row: str(row).replace("$", "").replace(",", "")).astype(float)
price_predict['extra_people'] = price_predict['extra_people'].apply(lambda row: str(row).replace("$", "").replace(",", "")).astype(float)

In [63]:
# lets create dummy columns to interpret categorical values
price_predict_obj = price_predict.select_dtypes(include='object')
price_predict_obj = pd.get_dummies(price_predict_obj, dummy_na=True, drop_first=True)
price_predict_obj.head()

,experiences_offered_nan,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,host_is_superhost_t,host_is_superhost_nan,host_has_profile_pic_t,host_has_profile_pic_nan,city_seattle,...,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,zipcode_99 98122,zipcode_nan
0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# lets concat numerical columns with dummy columns
price_predict = pd.concat([price_predict.select_dtypes(exclude='object'), price_predict_obj], axis=1)
price_predict.head()

,price,host_response_rate,host_acceptance_rate,accommodates,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,...,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,zipcode_99 98122,zipcode_nan
0,250.0,NaN,NaN,4,1.5,2.0,3.0,NaN,35.0,1,...,0,0,0,0,0,0,0,0,0,0
1,65.0,100.0,100.0,2,1.0,1.0,1.0,95.0,10.0,0,...,0,0,0,0,0,0,0,0,0,0
2,65.0,100.0,88.0,2,1.0,1.0,1.0,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0
3,75.0,100.0,50.0,4,1.0,1.0,2.0,100.0,50.0,2,...,0,0,0,0,0,0,0,0,0,1
4,79.0,100.0,100.0,2,1.5,1.0,2.0,NaN,15.0,1,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# lets check how many null values there are in our dataset
price_predict.isnull().sum()

price                                       0
host_response_rate                        994
host_acceptance_rate                     1244
accommodates                                0
bathrooms                                  30
bedrooms                                   16
beds                                       10
security_deposit                         4195
cleaning_fee                             2137
guests_included                             0
extra_people                                0
minimum_nights                              0
maximum_nights                              0
number_of_reviews                           0
review_scores_value                      1477
experiences_offered_nan                     0
host_response_time_within a day             0
host_response_time_within a few hours       0
host_response_time_within an hour           0
host_response_time_nan                      0
host_is_superhost_t                         0
host_is_superhost_nan             

In [66]:
# lets remove some additional columns once the null values are not so impacting
price_predict.dropna(subset=['review_scores_value', 'bathrooms', 'bedrooms', 'beds'], inplace=True)
price_predict.isnull().sum()

price                                       0
host_response_rate                        525
host_acceptance_rate                      690
accommodates                                0
bathrooms                                   0
bedrooms                                    0
beds                                        0
security_deposit                         3173
cleaning_fee                             1534
guests_included                             0
extra_people                                0
minimum_nights                              0
maximum_nights                              0
number_of_reviews                           0
review_scores_value                         0
experiences_offered_nan                     0
host_response_time_within a day             0
host_response_time_within a few hours       0
host_response_time_within an hour           0
host_response_time_nan                      0
host_is_superhost_t                         0
host_is_superhost_nan             

In [67]:
# and fill the remaining null values with 0 (which makes sense for these columns)
price_predict = price_predict.fillna(0)

In [68]:
# lets split the data into values (x) and target (y)
y = price_predict['price']
X = price_predict.drop('price', axis=1)

In [69]:
# lets view X
X.head()

,host_response_rate,host_acceptance_rate,accommodates,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,extra_people,...,zipcode_98133,zipcode_98134,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98177,zipcode_98178,zipcode_98199,zipcode_99 98122,zipcode_nan
1,100.0,100.0,2,1.0,1.0,1.0,95.0,10.0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,100.0,88.0,2,1.0,1.0,1.0,0.0,0.0,1,20.0,...,0,0,0,0,0,0,0,0,0,0
3,100.0,50.0,4,1.0,1.0,2.0,100.0,50.0,2,25.0,...,0,0,0,0,0,0,0,0,0,1
4,100.0,100.0,2,1.5,1.0,2.0,0.0,15.0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
5,100.0,95.0,2,1.0,1.0,1.0,0.0,30.0,1,0.0,...,0,0,0,0,0,0,0,0,0,0


### Train the model
To train the model, the model chosen was the XGBoost Regressor, which is a powerful model for regression. The SVR was tested also, but XGBoost was better comparing them about the r2_score

In [70]:
# import modeling libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import r2_score

# scaling features to XGBoost
sc_x = StandardScaler()
sc_y = StandardScaler()

#splitting dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = sc_x.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train.values.reshape(-1, 1))

regr = xgb.XGBRegressor(
                        n_estimators=150,
                        reg_lambda=1,
                        gamma=0,
                        max_depth=3
                       )

# training the model
regr.fit(X_train, y_train.ravel())

# verifying the goodness of fit for the model, using the r2_score, because it is an important metric to regressions
print("Training score: " + str(r2_score(sc_y.inverse_transform(y_train), sc_y.inverse_transform(regr.predict(X_train)))))

C:\Users\grego\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning:

Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.

C:\Users\grego\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning:

Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.

Training score: 0.8438546048373005


The R2 Score of 0.84 was pretty good, once we have some missing data, as 'square_feet' what should be really important

In [75]:
def format_results_as_dataframe(sc_y, y_real, y_pred):
    '''
    Format the results real and predicted ones into a dataframe, for posterior data vizualization

    Parameters:
    sc_y (StandardScaler): The fitted standardscaler object to do the inverse transform
    y_real (numpy array): The array with the real values for the variable
    y_pred (numpy array): The array with the predicted values for the variable

    Returns:
    Dataframe with two columns ('real' and 'predict') with the real and predicted values in them

    '''
    results_df = pd.DataFrame()
    results_df['real'] = sc_y.inverse_transform(y_real).reshape(-1,)
    results_df['predict'] = sc_y.inverse_transform(y_pred).reshape(-1,)

    return results_df

In [79]:
# Lets format the outputs as a dataframe and after, view some results
train_values = format_results_as_dataframe(sc_y=sc_y, y_real=y_train, y_pred=regr.predict(X_train))
train_values.head()

,real,predict
0,200.0,156.018951
1,255.0,209.333435
2,75.0,89.357765
3,650.0,647.546387
4,85.0,140.493103


In [80]:
# and plot the results
fig = px.scatter(train_values, x='real', y="predict")
fig.show()

In [81]:
# lets check r2 score for test dataset
print("Test score: " + str(r2_score(y_test, sc_y.inverse_transform(regr.predict(sc_x.transform(X_test))))))

C:\Users\grego\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning:

Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.

Test score: 0.6291758474625415


A R2 Score of 0.63 is not so good for the test dataset, which implies we had an overfitting. Additionally, as we could see for the plot, the results given by the estimator are a kind of good, but no wonderful.

In [86]:
# lets view some test results
# But first, lets format the data into a dataframe with our format_results_as_dataframe function
test_values = format_results_as_dataframe(sc_y=sc_y, y_real=sc_y.transform(y_test.values.reshape(-1, 1)), y_pred=regr.predict(sc_x.transform(X_test)))
test_values.head()

C:\Users\grego\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning:

Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.



,real,predict
0,75.0,81.967110
1,75.0,154.956223
2,70.0,67.276596
3,125.0,128.651031
4,125.0,119.731941


In [87]:
# lets view the test results
fig = px.scatter(test_values, x='real', y="predict")
fig.show()

In [88]:
# And, finally, plot our feature importances for the model
feature_importances = pd.DataFrame()
feature_importances['features'] = price_predict.drop('price', axis=1).columns
feature_importances['importance'] = regr.feature_importances_
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

In [89]:
fig = px.bar(feature_importances, x='features', y='importance')
fig.show()

In [90]:
# Top 10 of feature importances
feature_importances.head(10)

,features,importance
4,bedrooms,0.155995
23,city_seattle,0.124007
44,room_type_Private room,0.115984
3,bathrooms,0.041675
7,cleaning_fee,0.032186
59,zipcode_02116,0.029217
45,room_type_Shared room,0.028079
16,host_response_time_within a few hours,0.027093
120,zipcode_98199,0.022059
2,accommodates,0.020214


So, according to the table above, the most relevant features that impact in the price are the number of bedrooms, the type of bedroom and the city. The model accuracy was not the best, we got a kind of overfitting and it is necessary a further feature engineering and fine-tuning to improve our estimator. 